In [132]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline
# %pip install hf_xet
# import torch
# %pip install torch
%pip install readability-lxml


# # Descargar recursos necesarios de NLTK
nltk.download('punkt_tab')
nltk.download('gutenberg')
nltk.download('stopwords')

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [133]:
def generate_summary(article_text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    
    # BART tiene límite de tokens, cortar si el texto es muy largo
    article_text = article_text[:1024]
    
    summary = summarizer(article_text, max_length=130, min_length=50, do_sample=False)
    return summary[0]['summary_text']

In [134]:
def huggingface_classifier(data):
    from transformers import pipeline

    # Crear pipeline i definir model
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    # Important si el text es en anglés, posar els topics també en anglés

    # Definició dels topics
    # Definició dels temes que pot tenir l'article
    # possible_labels = ["política", "deportes", "economía", "ciencia", "noticias","tecnología", "salud", "cultura", "guerra", "internacional"]
    
    possible_labels = ["politics", "sports", "economy", "science", "technology", "health", "culture", "war", "international"]


    article_text = " ".join(data)[:1024]
    result = classifier(article_text, candidate_labels=possible_labels)

    print("Topic més probable:", result['labels'][0])
    print("Scores:", list(zip(result['labels'], result['scores'])))

    return result

In [135]:
def title_article_preprocessing_a(clean_text):

    # 1. Toeknize del texte
    # tokens = word_tokenize(clean_text, language='spanish')
    tokens = word_tokenize(clean_text, language='english')
    
    # 2. Normalitzar a minúscules i eliminar puntuació
    tokens = [t.lower() for t in tokens if t.isalpha()]  # solo palabras
    
    # 3. Eliminar stopweords
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words]
    
    # print(tokens[:50])

    return tokens


def title_article_preprocessing_b(clean_text):
    # ToDo: Implementar otra funcion de procesameinto de texto diferente 
    pass

In [136]:
import requests
from bs4 import BeautifulSoup
from readability import Document
from lxml import html

url = "https://link.springer.com/article/10.1186/s12915-020-00925-x"



def scrape_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 1. Intentar <article>
    article = soup.find("article")
    if article:
        text = article.get_text(separator="\n")
    else:
        # 2. Intentar <main>
        main = soup.find("main")
        if main:
            text = main.get_text(separator="\n")
        else:
            # 3. Fallback: todos los <p>, unir el bloque más largo
            paragraphs = soup.find_all("p")
            blocks = []
            current_block = []
            for p in paragraphs:
                current_block.append(p.get_text())
                # Separar bloques grandes por lógica simple
                if len(current_block) > 3:  
                    blocks.append("\n".join(current_block))
                    current_block = []
            if current_block:
                blocks.append("\n".join(current_block))
            # Elegir el bloque más largo (probable artículo)
            text = max(blocks, key=len) if blocks else ""
    
    # Limpiar líneas vacías
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    data = "\n".join(lines)

    return data


def scrape_text_refined(url):
    response = requests.get(url)
    response.raise_for_status()

    tree = html.fromstring(response.content)

    # Buscar el contenedor principal del artículo (para Springer suele ser c-article-body)
    article_divs = tree.xpath('//div[contains(@class,"c-article-body")]')
    
    if article_divs:
        paragraphs = article_divs[0].xpath('.//p//text()')
        data = "\n".join([p.strip() for p in paragraphs if p.strip()])
    else:
        # Fallback: todos los <p> de la página
        paragraphs = tree.xpath('//p//text()')
        data = "\n".join([p.strip() for p in paragraphs if p.strip()])


    return data


    


In [ ]:
def execution_1(url):
    data = scrape_text(url)
    tokens = title_article_preprocessing_a(data)
    results = huggingface_classifier(tokens)
    resume = generate_summary(data)
    print(f"Resum de l'article: {resume}")


def execution_2(url):
    data = scrape_text_refined(url)
    tokens = title_article_preprocessing_a(data)
    results = huggingface_classifier(tokens)
    resume = generate_summary(data)
    print(f"Resum de l'article: {resume}")


def execution_3(url):
    data = scrape_text(url)
    tokens = title_article_preprocessing_b(data)
    results = huggingface_classifier(tokens)
    resume = generate_summary(data)
    print(f"Resum de l'article: {resume}")


def execution_4(url):
    data = scrape_text(url)
    tokens = title_article_preprocessing_a(data)
    results = train_model(tokens)
    resume = generate_summary(data)
    print(f"Resum de l'article: {resume}")


def execution_5(url):
    data = scrape_text(url)
    tokens = title_article_preprocessing_a(data)
    results = huggingface_classifier(tokens)
    resume = generate_summary_b(data)
    print(f"Resum de l'article: {resume}")
    


In [138]:
def main():

    urls = ["https://link.springer.com/article/10.1186/s12915-020-00925-x"]

    for url in urls:
        print(f"############# Execution 1: URL{url}")
        execution_1(url)

        print(f"############# Execution 2: URL{url}")
        execution_2(url)



main()

############# Execution 1: URLhttps://link.springer.com/article/10.1186/s12915-020-00925-x


Device set to use cpu


Topic més probable: science
Scores: [('science', 0.6881455779075623), ('international', 0.07561494410037994), ('economy', 0.0668274536728859), ('culture', 0.0525495819747448), ('technology', 0.044419512152671814), ('health', 0.03472081199288368), ('war', 0.017296474426984787), ('sports', 0.01105914730578661), ('politics', 0.009366490878164768)]


Device set to use cpu


Resum de l'article: Comparative analysis of morabine grasshopper genomes reveals highly abundant transposable elements and rapidly proliferating satellite DNA repeats. The study was published in the journal MCB Biology. For confidential support call the Samaritans on 08457 90 90 90 or visit a local Samaritans branch, see www.samaritans.org for details.
############# Execution 2: URLhttps://link.springer.com/article/10.1186/s12915-020-00925-x


Device set to use cpu


Topic més probable: science
Scores: [('science', 0.5759952068328857), ('international', 0.10380233079195023), ('economy', 0.08592421561479568), ('culture', 0.08271900564432144), ('technology', 0.07284616678953171), ('health', 0.03962034359574318), ('war', 0.01820867508649826), ('politics', 0.010658523067831993), ('sports', 0.010225552134215832)]


Device set to use cpu


Resum de l'article: Repetitive DNA sequences, including transposable elements (TEs) and tandemly repeated satellite DNA (satDNAs), are found in high proportion in organisms across the Tree of Life. Grasshoppers have large genomes, averaging 9 Gb, that contain a high proportion of repetitive DNA, which has hampered progress in assembling reference genomes.
